In [23]:
import pandas as pd
import json

fileName = "2019 Happiness Data.csv"
happinessDF = pd.read_csv(fileName) 

fileName = "economic_freedom_index2019_data.csv"
economicFreedomDF = pd.read_csv(fileName,encoding='latin-1')

json_file = open("countries.geo.json", "r")
countries_geojson = json.load(json_file)

json_file = open("factbook-country-profiles.json", "r")
factbook = json.loads(json.dumps("factbook-country-profiles.json"))

TypeError: Object of type TextIOWrapper is not JSON serializable

In [16]:
factbook-country-profiles.json = factbook-country-profiles.json.replace("\'", "\"")
json_file = open("factbook-country-profiles.json", "r")
factbook = json.load(json_file)

SyntaxError: can't assign to operator (2775355129.py, line 1)

In [24]:
factbook = json.loads(json.dumps("factbook-country-profiles.json"))

In [26]:
String exampleRequest = FileUtils.readFileToString(new File("exampleJsonRequest.json"), StandardCharsets.UTF_8);
factbook

'factbook-country-profiles.json'

In [20]:
json.load(json_file)

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 3 (char 2)